# Save and load the model
In this section we will look at how to persist model state with saving, loading and running model predictions.


このセクションでは，モデル予測の保存，読み込み，実行を行なってモデルの状態を永続化する方法を見る．

In [1]:
import torch
import torch.onnx as onnx
import torchvision.models as models

## Saving and Loading Model Weights
PyTorch models store the learned parameters in an internal state dictionary, called `state_dict`.
These can be persisted via the `torch.save` method:

PyTorchモデルは，`state_dict`と言われる内部状態ディクショナリ内で学習したパラメータを保存する．
これらは`torch.save`メソッドを通して永続化できる．

In [2]:
# causing!
# vgg16 model has over 100M parameters.
# So the size of  'model_weights.pth' file is over 500 MB.
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/shibanotoshiki/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

To load model weights, you need to create an instance of the same model first, and then load the parameters using `load_state_dict()` method.


モデルの重みを読み込むために，最初にモデルのインスタンスの作成し，`laod_state_dict()`メソッドを用いてパラメータを読み込む．

In [3]:
model = models.vgg16() # we do not specify pretraind=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

#### NOTE
be sure to call `model.eval()` method before inferencing to set the dropout and batch normalization layers to evaluation mode.
Failing to do this will yield inconsistent inference results.


ドロップアウト層とバッチ正規化層を評価モードに設定するために，推論する前に必ず，`model.eval()`メソッドを呼ぶ．
これを怠ると，一貫性のない推論結果が得られる．

## Saving and Loading Models with Shapes
When loading model wights, we needed to instantiate the model class first, because the calss defines the structure of a network.
We might want to save the structure of this class together with the model, in which case we can pass `model` (and not `model.state_dict()`) to the saving function:



モデルの重みを読み込む時，クラスはネットワークの構造を定義するために，モデルクラスをインスタンス化する必要があった．
モデルと一緒にクラスの構造を保存したい場合は，`model.state_dict()`ではなく，`model`を保存関数に渡す．

In [4]:
torch.save(model, 'model.pth')

We can then load the model like this:


このようにモデルを読み込むことができる．

In [5]:
model = torch.load('model.pth')

#### NOTE
This approach uses Python [pickle](https://docs.python.org/3/library/pickle.html) module when serializing the model, thus it relies on the actual class definition to be abailable when loading the model.


このアプローチでは，モデルをシリアル化する時にPythonのpickleモジュールを使うため，モデルをロードする時に使用できる実際のクラス定義に依存する．


## Exporting Model to ONNX
PyTorch also has native ONNX export support.
Given the dynamic nature of the PyTorch execution graph, however, the export process must traverse the execution graph to produce a persisted ONNX model.
For this reason, a test variable of the appropriate size should be passed in to the export routine (in our case, we will create a dummy zero tensor of the correct size):



PyTorchはネイティブONNX出力もサポートしている．
しかしながら，PyTorch実行グラフのダイナミックな性質を考えると，永続化されたONNXモデルを生成するためには，出力過程は実行グラフを横切らなければならない．
このため，適切なサイズのテスト変数を出力ルーティンに渡す必要がある(この場合，正しいサイズのダミーのゼロtensorを渡す)．

In [7]:
input_image = torch.zeros((1, 3, 224, 224))
onnx.export(model, input_image, 'model.onnx')

There are a lot of things you can do with ONNX model, including running inference on different platforms and in different programming languages.
For more details, we recommend visiting [ONNX tutorial](https://github.com/onnx/tutorials).


様々なプラットフォームやプログラミング言語で推論できるなど，ONNXモデルでできることはたくさんある．
詳しくはONNXチュートリアルを訪れることをお勧めします．

Congratulations!
You have completed the PyTorch beginner tutorial!
Try [revisiting the first page](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) 
to see the tutorial in its entirety again.
We hope this tutorial has helped you get started with deep learning on PyTorch.
Good luck!

おめでとう！
PyTorch初心者チュートリアルは完了したよ！
もう一度最初のページ(Quickstard)に戻ってみよう．
このチュートリアルがあなたがPyTorchでDeep Learningを始めるのに役立つことを願っているよ．
頑張ろう！